# Preprocessing Second Batch of Songs

In [145]:
from functools import reduce
from glob import glob
import json
from multiprocessing import cpu_count
import pickle
import os
import re

from deepmerge import always_merger

import numpy as np
import pandas as pd

from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.decomposition import TruncatedSVD

## Table of Contents

1. Loading in Tables


## 1. Loading in Tables

In [3]:
# all songs I attempted to pull
with open('../data/new_tracks_20190103.json', 'r') as f:
    new_tracks = json.load(f)


matched_songs = pd.read_csv('../data/matched_songs_20190209.csv', index_col = 0)
main_wfeats = pd.read_csv('../data/main_wfeats.csv', index_col = 0)
genres = pd.read_csv('../data/genres.csv', index_col = 0)
gs_lookup = pd.read_csv('../data/gsong_lookup.csv', index_col=0)

In [149]:
main_wfeats.columns

Index(['song_id', 'album_release_date', 'artist_id', 'artist_name',
       'duration_ms', 'explicit', 'linked_album', 'song_title', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'key_changes', 'mean_song_conf', 'mean_loudness', 'mean_mode',
       'mean_mode_conf', 'mean_tempo', 'mean_tempo_conf', 'var_song_conf',
       'var_loudness', 'var_mode', 'var_mode_conf', 'var_tempo',
       'var_tempo_conf', 'tm_dim_1', 'tm_dim_2', 'tm_dim_3', 'tm_dim_4',
       'tm_dim_5', 'tm_dim_6', 'tm_dim_7', 'tm_dim_8', 'tm_dim_9', 'tm_dim_10',
       'tm_dim_11', 'tm_dim_12', 'tv_dim_1', 'tv_dim_2', 'tv_dim_3',
       'tv_dim_4', 'tv_dim_5', 'tv_dim_6', 'tv_dim_7', 'tv_dim_8', 'tv_dim_9',
       'tv_dim_10', 'tv_dim_11', 'tv_dim_12', 'pm_dim_1', 'pm_dim_2',
       'pm_dim_3', 'pm_dim_4', 'pm_dim_5', 'pm_dim_6', 'pm_dim_7', 'pm_dim_8',
       'pm_dim_9', 'pm_dim_10', 'pm_dim_

In [4]:
artists = pd.read_csv('../data/artist_list_v2.csv', index_col=0)

### 1a. Grabbing Mean & Variance Segment Values

In [5]:
def combine_section_mean_var(fp):
    '''
    Given a file path, grabs mean and variance segment dictionaries
    and combines them together
    '''
    directory = {
        'means' : [],
        'vars' : [],
    }
    
    # grabbing all files
    for file in os.listdir(fp):
        if 'section_mean_' in file:
            directory['means'].append(file)
        elif 'section_var_' in file:
            directory['vars'].append(file)
    
    # base files
    with open(f"{fp}/{directory['means'][0]}", 'r') as f:
        mean_dict = json.load(f)
    with open(f"{fp}/{directory['vars'][0]}", 'r') as f:
        vars_dict = json.load(f)
    
    # adding addtl files
    for i in range(1, len(directory['means'])):
        with open(f"{fp}/{directory['means'][i]}", 'r') as f:
            addition = json.load(f)
        mean_dict.extend(addition)
                  
    for i in range(1, len(directory['vars'])):
        with open(f"{fp}/{directory['vars'][i]}", 'r') as f:
            addition = json.load(f)
        vars_dict.extend(addition)
    
    return mean_dict, vars_dict

In [6]:
means, variances = combine_section_mean_var('../data')

In [7]:
len(variances)

651452

In [8]:
len(new_tracks)

651517

In [9]:
len(matched_songs)

159438

In [10]:
[value for value in means[0].values()][0]

{'confidence': 0.601625,
 'duration': 26.5366675,
 'loudness': -14.359750000000002,
 'mode': 0.875,
 'mode_confidence': 0.3652500000000001,
 'tempo': 121.35325,
 'tempo_confidence': 0.4575}

In [11]:
def unpack_lists(lst):
    '''
    Unpacks a list of dictionaries into a dictionary
    '''
    dct = {}
    for i in range(len(lst)):
        dct[[key for key in lst[i].keys()][0]] \
            = [value for value in lst[i].values()][0]
    return dct

In [12]:
mean_dict = unpack_lists(means)
var_dict = unpack_lists(variances)

In [13]:
def find_errors(dct):
    '''
    returns error messages if found in mean or var dicts
    '''
    error_dct = {}
    for i in dct.copy():
        if isinstance(dct[i], str):
            error_dct[i] = dct[i]
            del dct[i]
    return dct, error_dct

In [14]:
mean_dict, mean_errors = find_errors(mean_dict)

In [15]:
var_dict, var_errors = find_errors(var_dict)

In [16]:
len(mean_dict), len(var_dict)

(650263, 650263)

In [17]:
len(mean_errors), len(var_errors)

(1189, 1189)

In [18]:
mean_df = pd.DataFrame.from_dict(mean_dict, orient='index')

In [19]:
var_df = pd.DataFrame.from_dict(var_dict, orient='index')

In [151]:
mean_df.to_csv('../data/section_mean_df_20190413.csv')
var_df.to_csv('../data/section_var_df_20190413.csv')

In [20]:
mean_df.head()

,confidence,duration,loudness,mode,mode_confidence,tempo,tempo_confidence
0007aPK8VmXN4ycL2OcBFa,0.552778,32.807407,-7.355333,0.888889,0.354889,114.153000,0.286667
0008G8TW7eiVfwlRRsKlgW,0.549364,19.008486,-11.250909,0.909091,0.629455,133.749182,0.371545
000BqzNd7gRYnK6umzTNZX,0.657167,20.035555,-5.852000,0.833333,0.285667,145.424167,0.365333
000CSIqE1KcjAiZYYWXV18,0.477333,24.455557,-12.882333,0.166667,0.552833,124.057583,0.438833
000G1xMMuwxNHmwVsBdtj1,0.489167,15.195622,-6.957667,0.833333,0.262083,190.880667,0.272000


In [21]:
new_tracks['0008G8TW7eiVfwlRRsKlgW']

{'Song Title': 'Don`t Go', 'Artist': 'Stevie B'}

In [22]:
new_tracks_df = pd.DataFrame.from_dict(new_tracks, orient='index')

In [23]:
artists.reset_index(inplace=True)
artists.rename(columns= {'s_artist_id' : 'artist_id'},
               inplace= True)

In [24]:
new_tracks_df.head()

,Song Title,Artist
0007aPK8VmXN4ycL2OcBFa,Bodhisattva - Live,Toto
0008G8TW7eiVfwlRRsKlgW,Don`t Go,Stevie B
000BqzNd7gRYnK6umzTNZX,You Still Want Me - 2014 Remastered Version,The Kinks
000CSIqE1KcjAiZYYWXV18,Under The Sun (Ecclesiastes),Michael Card
000G1xMMuwxNHmwVsBdtj1,Will Anything Happen,Blondie


In [25]:
new_tracks_df.reset_index(inplace=True)

In [26]:
new_tracks_df.rename(columns={'index': 'track_id'}, inplace=True)

In [27]:
len(artists)

2426

### 1b. Adding in Artist IDs (and subsequently Genres into new tracks)

It appears as though I'm loosing ~72k tracks when completing the merge. My guess is that it's likely these songs have multiple artists tied to them, and I did not pull the artist that matched the listing I currently have.

In [28]:
tracks_w_artist_df = new_tracks_df.merge(artists, 
                                         left_on='Artist',
                                         right_on='name')

In [29]:
len(new_tracks_df)

651517

In [30]:
len(tracks_w_artist_df)

578062

In [31]:
tracks_w_artist_df.head()

,track_id,Song Title,Artist,artist_id,name,popularity,followers
0,0007aPK8VmXN4ycL2OcBFa,Bodhisattva - Live,Toto,0PFtn5NtBbbUNbU9EAmIWF,Toto,76,753405.0
1,00LEpojkN28l3W4axF8Ish,Don't Chain My Heart - Live,Toto,0PFtn5NtBbbUNbU9EAmIWF,Toto,76,753405.0
2,03IWDwx4slQdBg4lAt4H0n,Taint Your World (Live),Toto,0PFtn5NtBbbUNbU9EAmIWF,Toto,76,753405.0
3,042qp6lOUMoMHVT6R85u8W,Caught In the Balance - Live Version,Toto,0PFtn5NtBbbUNbU9EAmIWF,Toto,76,753405.0
4,06WhJ0TK0N9fyJFvAMRWEL,Girl Goodbye - Live,Toto,0PFtn5NtBbbUNbU9EAmIWF,Toto,76,753405.0


In [32]:
tracks_w_artist_df['name'].nunique()

2330

#### Looking at Matched Songs

In [33]:
matched_songs.head()

,artist_name,song_id,song_title,CID,PID,Title,Performer Name
0,robyn,6SluaPiV04KOaRTOIScoff,show me love,260151.0,14428.0,show me love,robyn
3,lukas graham,5kqIPrATaCc2LqxVWzQGbk,7 years,43137.0,53876.0,7 years,lukas graham
10,the killers,3aVyHFxRkf8lSjhWdJ68AW,just another girl,76427.0,39793.0,just another girl,the killers
11,tamia,0zIyxS6QxZogHOpGkI6IZH,deeper,8897.0,14338.0,deeper,tamia
14,kanye west,12D0n7hKpPcjuUpcbAKjjr,dont like.1,68936.0,2118.0,dont like.1,kanye west


In [34]:
len(matched_songs)

159438

#### Comparing Titles Against Matched Tracks -w- Artist IDs

In [35]:
matched_tracks_df = tracks_w_artist_df.merge(matched_songs,
                                        left_on='track_id',
                                        right_on='song_id')

In [36]:
matched_tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160200 entries, 0 to 160199
Data columns (total 14 columns):
track_id          160200 non-null object
Song Title        160200 non-null object
Artist            160200 non-null object
artist_id         160200 non-null object
name              160200 non-null object
popularity        160200 non-null int64
followers         160200 non-null float64
artist_name       160200 non-null object
song_id           160200 non-null object
song_title        160200 non-null object
CID               160200 non-null float64
PID               160200 non-null float64
Title             160200 non-null object
Performer Name    160200 non-null object
dtypes: float64(3), int64(1), object(10)
memory usage: 18.3+ MB


## 1c. Merging In Composer info to See Remaining Recommendable Songwriters

In [37]:
cw_lookup = pd.read_csv('../data/cw_lookup.csv', index_col=0)
writers = pd.read_csv('../data/writers.csv', index_col=0)

In [38]:
cw_lookup.head()

,CID,IPI
1,0,205786169
2,0,333982644
3,0,337563351
4,0,337563939
5,0,337564152


In [39]:
matched_w_CID = matched_tracks_df.merge(cw_lookup,
                                        on='CID')

In [40]:
matched_w_writers = matched_w_CID.merge(writers,
                                     on='IPI')

In [41]:
matched_w_writers.head()

,track_id,Song Title,Artist,artist_id,name,popularity,followers,artist_name,song_id,song_title,CID,PID,Title,Performer Name,IPI,Writer Name,PRO
0,0007aPK8VmXN4ycL2OcBFa,Bodhisattva - Live,Toto,0PFtn5NtBbbUNbU9EAmIWF,Toto,76,753405.0,toto,0007aPK8VmXN4ycL2OcBFa,bodhisattva,126127.0,6030.0,bodhisattva,toto,89150949,BECKER WALTER CARL,ASCAP
1,4F6419huGNh7rXB8Kr1rBf,Bodhisattva - Live,Toto,0PFtn5NtBbbUNbU9EAmIWF,Toto,76,753405.0,toto,4F6419huGNh7rXB8Kr1rBf,bodhisattva,126127.0,6030.0,bodhisattva,toto,89150949,BECKER WALTER CARL,ASCAP
2,6auUOYXyP76l4fiqC86c9E,Bodhisattva - Live,Toto,0PFtn5NtBbbUNbU9EAmIWF,Toto,76,753405.0,toto,6auUOYXyP76l4fiqC86c9E,bodhisattva,126127.0,6030.0,bodhisattva,toto,89150949,BECKER WALTER CARL,ASCAP
3,10JB78fwqaUrCFZj3XNn6L,Bodhisattva,Steely Dan,6P7H3ai06vU1sGvdpBwDmE,Steely Dan,68,601480.0,steely dan,10JB78fwqaUrCFZj3XNn6L,bodhisattva,126127.0,1699.0,bodhisattva,steely dan,89150949,BECKER WALTER CARL,ASCAP
4,6cdLlECd80adBM3fgEk9Km,Bodhisattva,Steely Dan,6P7H3ai06vU1sGvdpBwDmE,Steely Dan,68,601480.0,steely dan,6cdLlECd80adBM3fgEk9Km,bodhisattva,126127.0,1699.0,bodhisattva,steely dan,89150949,BECKER WALTER CARL,ASCAP


In [42]:
matched_w_writers['IPI'].nunique()

24672

In [43]:
matched_w_writers['Writer Name'].value_counts()[matched_w_writers['Writer Name'].value_counts() > 4].shape

(10939,)

In [152]:
matched_w_writers.to_csv('../data/matched_tracks_w_writers_20190413.csv')

In [44]:
matched_deduplicated_titles = matched_w_writers.drop_duplicates(subset=['Song Title', 'Artist', 'artist_name', 
                                                                        'song_title', 'CID', 'PID', 'Title',
                                                                        'Performer Name', 'IPI', 'Writer Name',
                                                                        'PRO'])

### 1d. Full Number of Recommended IPI's with Deduplicated Titles

Need to subtract 1 for the the `-` value. Though, perhaps I could match on same name `if -`.

In [45]:
matched_deduplicated_titles['IPI'].value_counts()[matched_deduplicated_titles['IPI'].value_counts() > 4].shape

(8974,)

Just looking at the writer names at the top and bottom. There's probably at least 10% aliases here

In [46]:
matched_deduplicated_titles['Writer Name'].nunique()

24519

In [47]:
matched_deduplicated_pre_writers = matched_deduplicated_titles.drop(columns=['IPI', 'Writer Name', 'PRO'])

In [48]:
matched_deduplicated_writers = matched_deduplicated_pre_writers.drop_duplicates()

### 1e. "Master" List of Recommendable Songs, Based on Spotify ID

In [49]:
matched_deduplicated_writers.shape

(107784, 14)

### 1f. Adding Pitch and Timbre Means / Variances

In [54]:
glob('../data/pitch_timbre_means_vars/*')

['../data/pitch_timbre_means_vars/']

In [93]:
with open('../data/pitch_timbre_means_vars/pitch_means_100000.json', 'r') as f:
    pm_test = json.load(f)
with open('../data/pitch_timbre_means_vars/pitch_var_100000.json', 'r') as f:
    pv_test = json.load(f)

In [94]:
pitch_test_dict = always_merger.merge(pm_test, pv_test)

In [78]:
from functools import reduce

In [150]:
def combine_pitch_timbre_files(file_path, output='dict'):
    '''
    Combines pitch and timbre mean & variance files
    into a single dictionary, or dataframe
    
    file_path : str, location of pitch and timbre files
    output : str, default 'dict' (can also pass 'df')
             outputs resulting file into either a dict
             or dataframe
    '''
    if not (output == 'dict') | (output == 'df'):
        raise ValueError('Output must either be set to dict or df')
    
    pitch_means_dict = {}
    pitch_var_dict = {}
    timbre_means_dict = {}
    timbre_var_dict = {}
    
    pitch_timbre_file_list = glob(f'{file_path}/*')

    print('Opening Files')
    for file_w_file_path in pitch_timbre_file_list:
        if 'pitch_var' in file_w_file_path:
            if len(pitch_var_dict) > 1:
                with open(f'{file_w_file_path}', 'r') as f:
                    temp_dict = json.load(f)
                    pitch_var_dict.update(temp_dict)
            else:
                with open(f'{file_w_file_path}', 'r') as f:
                    pitch_var_dict = json.load(f)
        
        elif 'pitch_means' in file_w_file_path:
            if len(pitch_means_dict) > 1:
                with open(f'{file_w_file_path}', 'r') as f:
                    temp_dict = json.load(f)
                    pitch_means_dict.update(temp_dict)
            else:
                with open(f'{file_w_file_path}', 'r') as f:
                    pitch_means_dict = json.load(f)
        
        elif 'timbre_var' in file_w_file_path:
            if len(timbre_var_dict) > 1:
                with open(f'{file_w_file_path}', 'r') as f:
                    temp_dict = json.load(f)
                    timbre_var_dict.update(temp_dict)
            else:
                with open(f'{file_w_file_path}', 'r') as f:
                    timbre_var_dict = json.load(f)        
        
        elif 'timbre_means' in file_w_file_path:
            if len(timbre_means_dict) > 1:
                with open(f'{file_w_file_path}', 'r') as f:
                    temp_dict = json.load(f)
                    timbre_means_dict.update(temp_dict)
            else:
                with open(f'{file_w_file_path}', 'r') as f:
                    timbre_means_dict = json.load(f)
    
    print('Organizing Dictionaries')
    for track_id in pitch_means_dict:
        pitch_means_dict[track_id] = {
            f'pm_{i}' : pitch_means_dict[track_id]['pitch_means'][i] \
            for i in range(len(pitch_means_dict[track_id]['pitch_means']))
        }
    for track_id in pitch_var_dict:
        pitch_var_dict[track_id] = {
            f'pv_{i}' : pitch_var_dict[track_id]['pitch_var'][i] \
            for i in range(len(pitch_var_dict[track_id]['pitch_var']))
        }
    for track_id in timbre_means_dict:
        timbre_means_dict[track_id] = {
            f'tm_{i}' : timbre_means_dict[track_id]['timbre_means'][i] \
            for i in range(len(timbre_means_dict[track_id]['timbre_means']))
        }
    for track_id in timbre_var_dict:        
        timbre_var_dict[track_id] = {
            f'tv_{i}' : timbre_var_dict[track_id]['timbre_var'][i] \
            for i in range(len(timbre_var_dict[track_id]['timbre_var']))
        }

    print('Merging Files')
    
    master_dict = reduce(lambda x, y: \
                         always_merger.merge(x, y), 
                         [pitch_means_dict,
                          pitch_var_dict,
                          timbre_means_dict,
                          timbre_var_dict])
    if output == 'dict':
        return master_dict
    else:
        return pd.DataFrame.from_dict(master_dict, 
                                      orient = 'index')

In [142]:
segment_dict = combine_pitch_timbre_files('../data/pitch_timbre_means_vars/')

Opening Files
Organizing Dictionaries
Merging Files


In [146]:
segment_df = pd.DataFrame.from_dict(segment_dict, orient='index')

In [148]:
segment_df.to_csv('../data/segment_df_20190413.csv')

### 1g. Adding in Genre Information

In [50]:
genres.head()

,genre
0,a cappella
1,acid house
2,acid jazz
3,acoustic blues
4,acoustic pop


#### 1f1. Creating Genre Song Lookup Table

In [53]:
# Should be earlier in the process

g_lookup = gs_lookup[['genre_id', 'genre_name', 'artist_id']]

In [54]:
g_lookup.head()

,genre_id,genre_name,artist_id
gs_id,,,
1,139,dance pop,6UE7nl9mha6s8z0wFQFIZ2
2,189,electropop,6UE7nl9mha6s8z0wFQFIZ2
3,196,europop,6UE7nl9mha6s8z0wFQFIZ2
4,404,pop,6UE7nl9mha6s8z0wFQFIZ2
5,410,pop rock,6UE7nl9mha6s8z0wFQFIZ2


In [ ]:
g_lookup.

In [55]:
matched_tracks_df.columns

Index(['track_id', 'Song Title', 'Artist', 'artist_id', 'name', 'popularity',
       'followers', 'artist_name', 'song_id', 'song_title', 'CID', 'PID',
       'Title', 'Performer Name'],
      dtype='object')

In [56]:
matched_w_genres_temp = matched_tracks_df.merge(g_lookup,
                                                on='artist_id')

In [57]:
matched_w_genres_temp.shape

(9941474, 16)

In [58]:
matched_w_genres_temp.columns

Index(['track_id', 'Song Title', 'Artist', 'artist_id', 'name', 'popularity',
       'followers', 'artist_name', 'song_id', 'song_title', 'CID', 'PID',
       'Title', 'Performer Name', 'genre_id', 'genre_name'],
      dtype='object')

In [59]:
gs_lookup = matched_w_genres_temp[['track_id', 
                                   'genre_id']]

In [61]:
gs_lookup.to_csv('../data/gsong_lookup_20190412.csv')

In [125]:
del matched_w_genres_temp

In [126]:
gs_lookup.head()

,track_id,genre_id
0,0007aPK8VmXN4ycL2OcBFa,8
1,0007aPK8VmXN4ycL2OcBFa,328
2,0007aPK8VmXN4ycL2OcBFa,448
3,0007aPK8VmXN4ycL2OcBFa,481
4,0007aPK8VmXN4ycL2OcBFa,8


#### 1f2. Creating Genre Dummies

In [1]:
import dask
import dask.dataframe as dd
from dask_ml.preprocessing import DummyEncoder
import pandas as pd

In [2]:
gs_lookup = pd.read_csv('../data/gsong_lookup_20190412.csv', index_col = 0)

/Users/jonjohnson/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
gs_lookup.genre_id = gs_lookup.genre_id.astype('category')

In [4]:
gs_lookup_ddf = dd.from_pandas(gs_lookup, 6)

In [5]:
de = DummyEncoder()

In [ ]:
genre_dummies = de.fit_transform(gs_lookup)

#### 1f3. SVD on Genres

In [113]:
tSVD = TruncatedSVD(50)
SVDg = tSVD.fit_transform(genre_dummies)

ValueError: n_components must be < n_features; got 50 >= 1